# 1. MODULO

In [1]:
# Importa el módulo 'os' para interactuar con el sistema de archivos y rutas del sistema operativo
import os

# Importa 'pandas' como 'pd', una biblioteca potente para manipulación y análisis de datos mediante DataFrames
import pandas as pd

# Importa el módulo 'datetime' para trabajar con fechas y horas de manera eficiente
import datetime

# 2. RUTAS

In [2]:
R_Maestro = r"C:\Users\anthonycaicedo\OneDrive - 891856000_CAPRESOCA E P S\Escritorio\BaseD\Maestros"
Maestro_Salidas = "Maestro.txt"
R_SALIDA = Fr"C:\Users\anthonycaicedo\OneDrive - 891856000_CAPRESOCA E P S\Escritorio\BaseD\{Maestro_Salidas}"

# 3. UNIFICACION

In [8]:
# Lista de encabezados proporcionados
encabezados = [
    "AFL_ID",
    "ENT_ID",
    "TPS_IDN_ID_CF",
    "HST_IDN_NUMERO_IDENTIFICACION_CF",
    "TPS_IDN_ID",
    "HST_IDN_NUMERO_IDENTIFICACION",
    "AFL_PRIMER_APELLIDO",
    "AFL_SEGUNDO_APELLIDO",
    "AFL_PRIMER_NOMBRE",
    "AFL_SEGUNDO_NOMBRE",
    "AFL_FECHA_NACIMIENTO",
    "TPS_GNR_ID",
    "AFL_PAIS_NACIMIENTO",
    "AFL_MUNICIPIO_NACIMIENTO",
    "AFL_NACIONALIDAD",
    "AFL_SEXO_IDENTIFICACION",
    "AFL_DISCAPACIDAD",
    "TPS_AFL_ID",
    "TPS_PRN_ID",
    "TPS_GRP_PBL_ID",
    "TPS_NVL_SSB_ID",
    "NUMEROFICHASISBEN",
    "TPS_CND_BNF_ID",
    "DPR_ID",
    "MNC_ID",
    "ZNS_ID",
    "AFL_FECHA_AFILIACION_SGSSS",
    "AFC_FECHA_INICIO",
    "NUMERO CONTRATO",
    "FECHADE INICIO DEL CONTRATO",
    "CNT_AFL_TPS_GRP_PBL_ID",
    "CNT_AFL_TPS_PRT_ETN_ID",
    "TPS_MDL_SBS_ID",
    "TPS_EST_AFL_ID",
    "CND_AFL_FECHA_INICIO",
    "CND_AFL_FECHA_INICIO_2",
    "GRP_FML_COTIZANTE_ID",
    "PORTABILIDAD",
    "COD_IPS_P",
    "MTDLG_G_P",
    "SUB_SISBEN_IV",
    "MARCASISBENIV+MARCASISBENIII",
    "CRUCE_BDEX_RNEC",
]

# Buscar todos los archivos .txt en subcarpetas de R_Maestro
txt_files = []
for root, dirs, files in os.walk(R_Maestro):
    for file in files:
        if file.lower().endswith('.txt'):
            txt_files.append(os.path.join(root, file))

# Leer y consolidar los archivos
dfs = []
for file_path in txt_files:
    file_name = os.path.basename(file_path)
    # Extraer la fecha del nombre del archivo
    try:
        idx = file_name.upper().find('EPSC25MC00')
        if idx != -1:
            fecha_str = file_name[idx+10:idx+18]  # 8 caracteres después de EPSC25MC00
            fecha_formateada = f"{fecha_str[:2]}/{fecha_str[2:4]}/{fecha_str[4:]}"
        else:
            fecha_formateada = ""
    except Exception:
        fecha_formateada = ""
    
    # Leer el archivo
    df = pd.read_csv(
        file_path,
        sep=",",
        header=None,
        encoding='ansi',
        dtype=str
    )
    
    # Verificar si el archivo tiene 38 o 43 columnas
    if df.shape[1] == 38:
        # Ajustar las columnas para que coincidan con el formato de 43 columnas
        for i in range(12, 17):
            df.insert(i, f"columna_vacia_{i}", "")  # Insertar columnas vacías
        # Renombrar las columnas para que coincidan con los encabezados
        df.columns = encabezados
    elif df.shape[1] == 43:
        # Renombrar las columnas directamente
        df.columns = encabezados
    else:
        # Si el archivo no tiene 38 o 43 columnas, omitirlo
        print(f"Archivo omitido por número inesperado de columnas: {file_path}")
        continue
    
    # Agregar la columna de fecha
    df['fecha_maestro'] = fecha_formateada
    dfs.append(df)

# Concatenar todos los DataFrames
df_maestro = pd.concat(dfs, ignore_index=True)

# Guardar el resultado en la ruta de salida
df_maestro.to_csv(R_SALIDA, sep=",", index=False, encoding='ansi')